# **TUIA - Procesamiento de Imágenes y Visión por Computadora (IA52)**
# **Trabajo Práctico Final**
### **Ejercicio 4 - Evaluación**
<br>

### *Alumno: Miguel Mussi*
### *Año: 2024*

---------------------
## **Tabla de contenidos**
1.   [**Librerías**](#)
2.   [**Ajustes Iniciales**](#)
3.   [**Optimización de inferencia**](#)
4.   [**Inferencia sobre las imágenes y cálculo del envido**](#)
        1.   [*Inferencia con CPU*](#)
        2.   [*Inferencia con GPU - Sin TensorRT*](#)
        3.   [*Inferencia con GPU - Con TensorRT*](#)
        4.   [*Comparativa de Inferencias*](#)
5.   [**Guardamos las imagenes con las predicciones**](#)
6.   [**Escritura del archivo envido.json**](#)
7.   [**Exportar directorios**](#)

## 1. **Librerías**

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 552.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 kB 5.3 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install onnx

In [3]:
import os
import cv2
import time
import json
import torch
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. **Ajustes Iniciales**

In [35]:
# Nombre del alumno
student_name = 'miguel_mussi'

# Ruta al directorio principal
main_path = '/kaggle/input/cv-tp-final/CV_TP_Final'

# Ruta al archivo de pesos
model_path = os.path.join(main_path, 'model/weights/best.pt')
# model_path_tensor = os.path.join(main_path, '/kaggle/working/model/weights/best.onnx')
model_path_tensor = os.path.join('/kaggle/working', 'best.onnx')

# Ruta al directorio que contiene las imagenes
imgs_dir = os.path.join(main_path, 'data/eval/images/val')

# Ruta al directorio de destino de las detecciones
base_dir = os.path.join('/kaggle/working/data/out/')
dets_dir = os.path.join(base_dir, student_name)

# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)
os.makedirs(dets_dir)

In [61]:
# Nombre del alumno
student_name = 'miguel_mussi'

# Ruta al directorio principal
main_path = '/kaggle/input/cv-tp-final/CV_TP_Final'

# Ruta del directorio de trabajo
working_dir = '/kaggle/working/'

# Ruta al archivo de pesos copiado en el directorio de trabajo
model_path_working = os.path.join(working_dir, 'model/weights/best.pt')

# Copiar la carpeta 'model' al directorio de trabajo
if not os.path.exists(os.path.join(working_dir, 'model')):
    shutil.copytree(os.path.join(main_path, 'model'), os.path.join(working_dir, 'model'))

# Ruta para guardar el modelo exportado
model_path_tensor = os.path.join(working_dir, 'model/weights/best.onnx')

# Ruta al directorio que contiene las imágenes
imgs_dir = os.path.join(main_path, 'data/eval/images/val')

# Ruta al directorio de destino de las detecciones
base_dir = os.path.join(working_dir, 'data/out/')
dets_dir = os.path.join(base_dir, student_name)

# Reestablecimiento del directorio de destino (eliminación)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)
os.makedirs(dets_dir)



In [55]:
if os.path.exists('/kaggle/working/model'):
    shutil.rmtree('/kaggle/working/model')
# os.makedirs('/kaggle/working/model')

In [9]:
# Seteo de tiempos de ejecución
execution_time = {'cpu': 0, 'gpu': 0, 'rt': 0}

## 3. **Optimización de inferencia**

In [57]:
# Cargamos el modelo
model = YOLO(model_path_working)

In [62]:
# Lo exportamos como un tensor para utilizar tensorRT con la GPU
model.export(format='onnx', imgsz=640, dynamic=True)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 268 layers, 43,644,387 parameters, 0 gradients, 165.0 GFLOPs

PyTorch: starting from '/kaggle/working/model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 53, 8400) (83.6 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 3.8s, saved as '/kaggle/working/model/weights/best.onnx' (166.6 MB)

Export complete (8.0s)
Results saved to /kaggle/working/model/weights
Predict:         yolo predict task=detect model=/kaggle/working/model/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/model/weights/best.onnx imgsz=640 data=/kaggle/working/dataset.yaml  
Visualize:       https://netron.app


'/kaggle/working/model/weights/best.onnx'

### Comprobación de entorno gráfico

In [64]:
print(torch.cuda.is_available())

True


In [19]:
# print(f'Usando el dispositivo: {device}')

In [18]:
print(f'Dispositivo del modelo: {next(model.parameters()).device}')

Dispositivo del modelo: cpu


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118




























In [ ]:
# device_id = 0  # El ID de dispositivo que deseas usar
# if device_id < torch.cuda.device_count():
#     device = torch.device(f'cuda:{device_id}')
# else:
#     raise AssertionError("Invalid device id")

In [65]:
num_gpus = torch.cuda.device_count()
print(f'Número de GPUs disponibles: {num_gpus}')

Número de GPUs disponibles: 0


## 4. **Inferencia sobre las imágenes y cálculo del envido**

In [22]:
def calcular_envido(num1,num2):
    n1 = int(num1)
    n2 = int(num2)
    ret = n1 if n1<10 else 0
    ret += n2 if n2<10 else 0
    return ret + 20

### 4.1. *Inferencia con CPU*

In [24]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            if num not in card_types[palo]:
                card_types[palo].append(num)
            else:
                print("Se encontraron cartas repetidas")
                invalid_cards += 1

    envido = {}

    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3 or invalid_cards:
        print('La cantidad de cartas no permite calcular el envido.\n')

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo

        # calculamos el envido
        # Iteramos en el diccionario - Key = palos / Value: números
        for key, values in card_types.items():
            envido[key]=0
            # Dos cartas del mismo palo
            if len(values) == 1:
                envido[key] = int(num) if len(num)==1 else 0

            elif len(values) == 2:
                envido[key] = calcular_envido(values[0],values[1])

            # Tres cartas del mismo palo
            elif len(values) == 3:
                print('Mano con Flor.\n')
                envido[key] = calcular_envido(values[0],values[1])
                envido[key] = max(envido[key],calcular_envido(values[0],values[2]))
                envido[key] = max(envido[key],calcular_envido(values[1],values[2]))


    mejor_palo = 'N/A'
    mejor_punto = 0

    for palo, punto in envido.items():
        if punto > mejor_punto:
            mejor_punto=punto
            mejor_palo = palo

    # Almacenamos los resultados del envido en el diccionario para el JSON


    card_js_file[img_filename] = {
            'total_cards': len(filtered_cards),
            'cards': card_types,
            'points': mejor_punto,       # Puntos de envido
            'figure': mejor_palo          # Palo
        }


# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['cpu'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['cpu']:.2f} segundos")


image 1/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 960.6ms
La cantidad de cartas no permite calcular el envido.

image 2/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 936.6ms
image 3/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 947.4ms
La cantidad de cartas no permite calcular el envido.

image 4/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 947.2ms
image 5/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 924.7ms
La cantidad de cartas no permite calcular el envido.

image 6/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173836085.jpg: 640x512 1 6B, 1 7E, 919.5ms
La cantidad de cartas no permite calcular el 

### 4.2. *Inferencia con GPU - Sin TensorRT*

In [26]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
# device = torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            if num not in card_types[palo]:
                card_types[palo].append(num)
            else:
                print("Se encontraron cartas repetidas")
                invalid_cards += 1

    envido = {}

    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3 or invalid_cards:
        print('La cantidad de cartas no permite calcular el envido.\n')

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo

        # calculamos el envido
        # Iteramos en el diccionario - Key = palos / Value: números
        for key, values in card_types.items():
            envido[key]=0
            # Dos cartas del mismo palo
            if len(values) == 1:
                envido[key] = int(num) if len(num)==1 else 0

            elif len(values) == 2:
                envido[key] = calcular_envido(values[0],values[1])

            # Tres cartas del mismo palo
            elif len(values) == 3:
                print('Mano con Flor.\n')
                envido[key] = calcular_envido(values[0],values[1])
                envido[key] = max(envido[key],calcular_envido(values[0],values[2]))
                envido[key] = max(envido[key],calcular_envido(values[1],values[2]))


    mejor_palo = 'N/A'
    mejor_punto = 0

    for palo, punto in envido.items():
        if punto > mejor_punto:
            mejor_punto=punto
            mejor_palo = palo

    # Almacenamos los resultados del envido en el diccionario para el JSON


    card_js_file[img_filename] = {
            'total_cards': len(filtered_cards),
            'cards': card_types,
            'points': mejor_punto,       # Puntos de envido
            'figure': mejor_palo          # Palo
        }


# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['gpu'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['gpu']:.2f} segundos")


image 1/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 48.1ms
La cantidad de cartas no permite calcular el envido.

image 2/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 47.3ms
image 3/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 47.3ms
La cantidad de cartas no permite calcular el envido.

image 4/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 47.2ms
image 5/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 47.3ms
La cantidad de cartas no permite calcular el envido.

image 6/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173836085.jpg: 640x512 1 6B, 1 7E, 47.3ms
La cantidad de cartas no permite calcular el envido

### 4.3. *Inferencia con GPU - Con TensorRT*

In [68]:
# Cargamos el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Marca de tiempo antes de la inferencia
start_time = time.time()

# Ejecutamos la inferencia
results = YOLO(model_path_tensor).predict(source=imgs_dir, device=device)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Umbral de confianza para las detecciones
detection_threshold = 0.5

# Iteramos sobre los resultados (archivos)
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Filtrar detecciones con umbral de confianza inferior a 0.45
    filtered_cards = [box for box in result.boxes if box.conf.item() >= detection_threshold]

    # Procesamos cada carta detectada
    for card in filtered_cards:
        # Obtenemos el nombre de la carta
        # name = filtered_cards.names[int(card.boxes.cls)]
        name = cards.names[int(card.cls)]

        # Separamos el número y el palo de la carta
        if len(name) == 3:      # (Carta negra. Ejemplo 10E - 11E - 12B )
            num = name[0] + name[1]  # número de la carta negra
            palo = name[2]           # palo de la carta
        elif len(name) == 2:                   # (Carta blanca. Ejemplo xO - xC - xE - xB )
            num = name[0]            # número de la carta
            palo = name[1]           # palo de la carta
        else:
            num = name[0]            # número del comodín
            palo = 'N/A'             # palo del comodín

        # Comprobamos si la carta es inválida
        if num in ['8', '9', 'J']:
            invalid_cards += 1

        # Agregamos la carta al diccionario correspondiente
        if palo in card_types:
            if num not in card_types[palo]:
                card_types[palo].append(num)
            else:
                print("Se encontraron cartas repetidas")
                invalid_cards += 1

    envido = {}

    # Comprobar si la cantidad de cartas es 3
    if len(filtered_cards) != 3 or invalid_cards:
        print('La cantidad de cartas no permite calcular el envido.\n')

    # 3 cartas en la detección
    else:
        # Inicializamos la variable palo

        # calculamos el envido
        # Iteramos en el diccionario - Key = palos / Value: números
        for key, values in card_types.items():
            envido[key]=0
            # Dos cartas del mismo palo
            if len(values) == 1:
                envido[key] = int(num) if len(num)==1 else 0

            elif len(values) == 2:
                envido[key] = calcular_envido(values[0],values[1])

            # Tres cartas del mismo palo
            elif len(values) == 3:
                print('Mano con Flor.\n')
                envido[key] = calcular_envido(values[0],values[1])
                envido[key] = max(envido[key],calcular_envido(values[0],values[2]))
                envido[key] = max(envido[key],calcular_envido(values[1],values[2]))


    mejor_palo = 'N/A'
    mejor_punto = 0

    for palo, punto in envido.items():
        if punto > mejor_punto:
            mejor_punto=punto
            mejor_palo = palo

    # Almacenamos los resultados del envido en el diccionario para el JSON


    card_js_file[img_filename] = {
            'total_cards': len(filtered_cards),
            'cards': card_types,
            'points': mejor_punto,       # Puntos de envido
            'figure': mejor_palo          # Palo
        }


# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time['rt'] = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
# print(f'Tiempo de ejecución: {execution_time['cpu']:.2f} segundos')
print(f"Tiempo de ejecución: {execution_time['rt']:.2f} segundos")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /kaggle/working/model/weights/best.onnx for ONNX Runtime inference...



2024-07-31 18:11:56.299308943 [E:onnxruntime:Default, provider_bridge_ort.cc:1745 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1426 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2024-07-31 18:11:56.299339532 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:895 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirementsto ensure all dependencies are met.


image 1/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x640 1 4O, 1 11C, 788.8ms
image 2/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x640 1 1C, 1 2B, 1 5C, 823.2ms
image 3/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x640 1 1C, 1 2B, 812.0ms
image 4/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x640 1 3B, 1 6B, 1 7E, 795.0ms
image 5/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x640 1 3B, 1 6B, 1 7E, 1 J, 785.2ms
image 6/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173836085.jpg: 640x640 1 6B, 1 7E, 783.6ms
image 7/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_173859456.jpg: 640x640 1 3B, 790.9ms
image 8/32 /kaggle/input/cv-tp-final/CV_TP_Final/data/eval/images/val/IMG_20240630_17391

### 4.4. *Comparativa de Inferencias*

In [69]:
# Comparación de los tiempos de ejecución
print("Comparación de Tiempos de Ejecución:")
print(f"Tiempo de ejecución (CPU): {execution_time['cpu']:.2f} segundos")
print(f"Tiempo de ejecución (GPU): {execution_time['gpu']:.2f} segundos")
print(f"Tiempo de ejecución (TensorRT): {execution_time['rt']:.2f} segundos")

Comparación de Tiempos de Ejecución:
Tiempo de ejecución (CPU): 34.64 segundos
Tiempo de ejecución (GPU): 6.27 segundos
Tiempo de ejecución (TensorRT): 32.63 segundos


## 5. **Guardamos las imagenes con las predicciones**

In [ ]:
# Cargamos el modelo
# model = YOLO(model_path)              # Con CPU
model = YOLO(model_path).to(device)     # Con GPU (sin TensorRT)
# model = YOLO(model_path_tensor)       # Con GPU (con TensorRT)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Directorio donde van a ir las deteciones
detection_dir = f'{base_dir}/detections/'

# Creamos el directorio si no existe
os.makedirs(detection_dir, exist_ok=True)

# Iteramos sobre los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Guardamos la detección
    result.save(filename=f'{detection_dir}/{img_filename}')



image 1/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173711753_HDR.jpg: 640x512 1 4O, 1 11C, 1805.6ms

image 2/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173737533.jpg: 640x512 1 1C, 1 2B, 1 5C, 1304.6ms

image 3/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173758691.jpg: 640x512 1 1C, 1 2B, 1376.4ms

image 4/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173822184_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1357.8ms

image 5/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 1 J, 1363.4ms

image 6/32 /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/eval/images/val/

## 6. **Escritura del archivo envido.json**

In [71]:
with open(os.path.join(dets_dir, 'envido.json'), 'w') as jf:
    json.dump(card_js_file, jf, indent=4)

## 7. **Exportar directorios**

In [28]:
# Especifica la carpeta a comprimir y el nombre del archivo zip
folder_path = '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections'

zip_name = '/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip'

# Crear el archivo zip
shutil.make_archive(zip_name.replace('.zip', ''), 'zip', folder_path)

print(f"Archivo zip creado: {zip_name}")

Archivo zip creado: /content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip


In [27]:
# Eliminar un archivo
os.remove('/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections.zip')
print("Eliminación completa!")

Eliminación completa!


In [ ]:
# Eliminar un directorio y todo su contenido
shutil.rmtree('/content/drive/MyDrive/UNR/5 - Proc de Imág y Visión Comp. (IA52)/CV_TP_Final/data/out/detections')
print("Eliminación completa!")

Eliminación completa!
